In [64]:
import pandas as pd
top50_df = pd.read_csv("top10s_2010_2019.csv", encoding='latin-1')
top50_df.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [65]:
import numpy as np
from pathlib import Path
from collections import Counter

In [66]:
#Remove uneccessary columns
top50_df.drop(['Unnamed: 0', 'title', 'artist', 'top genre'],  inplace=True, axis=1)
top50_df.head()

,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,2010,97,89,67,-4,8,80,217,19,4,83
1,2010,87,93,75,-5,52,64,263,24,23,82
2,2010,120,84,76,-3,29,71,200,10,14,80
3,2010,119,92,70,-4,8,71,295,0,4,79
4,2010,109,84,64,-5,9,43,221,2,4,78


In [67]:
##Can't get it to add this column...

#Seperating Popular songs to >85 (Y) and <=84 (N (less popular))    
top50_df['Pop Y/N'] = pd.Series(np.where(top50_df.pop.values <= 85, 0, 1), top50_df.index)
top50_df.head()

AttributeError: 'function' object has no attribute 'values'

In [68]:
X = top50_df.drop("pop", axis=1)

y = top50_df["pop"]

In [69]:
#balance of 'Pop Y/N' values
y.value_counts()

72    33
78    27
73    26
76    24
65    23
      ..
48     1
27     1
35     1
7      1
39     1
Name: pop, Length: 71, dtype: int64

In [70]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Naive Random Oversampling

In [48]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state = 1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#View the Count
Counter(y_resampled)

Counter({72: 25,
         71: 25,
         78: 25,
         59: 25,
         41: 25,
         67: 25,
         63: 25,
         69: 25,
         85: 25,
         18: 25,
         73: 25,
         65: 25,
         76: 25,
         57: 25,
         36: 25,
         64: 25,
         61: 25,
         75: 25,
         51: 25,
         81: 25,
         77: 25,
         38: 25,
         79: 25,
         44: 25,
         84: 25,
         68: 25,
         88: 25,
         74: 25,
         92: 25,
         66: 25,
         25: 25,
         80: 25,
         52: 25,
         56: 25,
         31: 25,
         46: 25,
         82: 25,
         60: 25,
         45: 25,
         50: 25,
         27: 25,
         83: 25,
         34: 25,
         0: 25,
         70: 25,
         43: 25,
         62: 25,
         58: 25,
         53: 25,
         93: 25,
         54: 25,
         55: 25,
         35: 25,
         30: 25,
         29: 25,
         28: 25,
         86: 25,
         49: 25,
         87: 25

In [49]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state =1)
model.fit(X_resampled, y_resampled)

C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [50]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(y_test, y_pred))

0.005208333333333333


C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1854: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')


In [51]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [71]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      0.99      0.00      0.00      0.00         1
          7       0.00      0.00      0.99      0.00      0.00      0.00         0
         25       0.00      0.00      0.98      0.00      0.00      0.00         0
         29       0.00      0.00      0.99      0.00      0.00      0.00         0
         30       0.00      0.00      0.97      0.00      0.00      0.00         0
         31       0.00      0.00      0.97      0.00      0.00      0.00         0
         33       0.00      0.00      1.00      0.00      0.00      0.00         1
         34       0.00      0.00      0.97      0.00      0.00      0.00         1
         35       0.00      0.00      0.97      0.00      0.00      0.00         0
         36       0.00      0.00      0.99      0.00      0.00      0.00         2
         37       0.00      0.00      0.99      0.00      0.00      0.00         0
   

C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SMOTE Oversampling

In [72]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

Counter(y_resampled)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 4, n_neighbors = 6

# Balanced Random Forest Classifier

In [73]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
rf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [74]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

0.028645833333333332


C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1854: UserWarning: y_pred contains classes not in y_true
  warnings.warn('y_pred contains classes not in y_true')
